In [1]:
import random
import queue
# чисто задание всё как в бамажке
class Task:
    def __init__(self, task_id, priority, service_intensity):
        self.task_id = task_id
        self.priority = priority
        self.service_intensity = service_intensity

def simulate():
    arrival_intensities = [0.6, 0.7, 0.6, 0.3]
    cpu_intensities = [0.7, 0.5, 0.4, 0.3]
    gpu_intensities = [1.4, 1.2, 0.8, 0.4]
    total_time = 1000

    queue_capacity = 30
    task_queue = queue.Queue(queue_capacity)

    tasks = []
    for i in range(len(arrival_intensities)):
        tasks.append(
            Task(i + 1, i + 1, arrival_intensities[i])
        )

    cpu_tasks_serviced = 0
    gpu_tasks_serviced = 0
    total_tasks_serviced = 0

    max_queue_length = 0
    total_queue_length = 0
    queue_empty_count = 0

    current_time = 0
    while current_time < total_time:
        for task in tasks:
            if random.random() < task.service_intensity:
                if task_queue.qsize() < queue_capacity:
                    task_queue.put(task)
                    total_queue_length += task_queue.qsize()
                    max_queue_length = max(max_queue_length, task_queue.qsize())
                else:
                    print("Превышение очереди. Задание отклонено.")

        if not task_queue.empty():
            current_task = task_queue.get()
            if current_task.priority == 1 or current_task.priority == 4:
                cpu_tasks_serviced += 1
                service_time = random.expovariate(cpu_intensities[current_task.priority - 1])
            else:
                gpu_tasks_serviced += 1
                service_time = random.expovariate(gpu_intensities[current_task.priority - 1])

            current_time += service_time
        else:
            current_time += 1
            queue_empty_count += 1

    total_tasks_serviced = cpu_tasks_serviced + gpu_tasks_serviced
    cpu_utilization = cpu_tasks_serviced / total_tasks_serviced
    gpu_utilization = gpu_tasks_serviced / total_tasks_serviced

    avg_queue_length = total_queue_length / (total_time - queue_empty_count)

    return {
        "cpu_tasks_serviced": cpu_tasks_serviced,
        "gpu_tasks_serviced": gpu_tasks_serviced,
        "total_tasks_serviced": total_tasks_serviced,
        "cpu_utilization": cpu_utilization,
        "gpu_utilization": gpu_utilization,
        "max_queue_length": max_queue_length,
        "avg_queue_length": avg_queue_length
    }

# Вызываем функцию simulate() 10 раз и сохраняем результаты
results = []
for _ in range(10):
    results.append(simulate())

# Выводим результаты моделирования
for result in results:
    print("Результаты моделирования:")
    print(f"Задания для CPU: {result['cpu_tasks_serviced']}")
    print(f"Задания для GPU: {result['gpu_tasks_serviced']}")
    print(f"Всего обслуженных заданий: {result['total_tasks_serviced']}")
    print(f"Загрузка CPU: {result['cpu_utilization']:.2f}")
    print(f"Загрузка GPU: {result['gpu_utilization']:.2f}")
    print(f"Максимальная длина очереди: {result['max_queue_length']}")
    print(f"Средняя длина очереди: {result['avg_queue_length']:.2f}")
    print()

Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задание отклонено.
Превышение очереди. Задан

In [43]:
#слишком гениальное
import random
import simpy

class Task:
    def __init__(self, task_id, service_intensity):
        self.task_id = task_id
        self.service_intensity = service_intensity


def task_generator(env, arrival_intensities, task_queue):
    tasks = []
    for i in range(len(arrival_intensities)):
        tasks.append(Task(i + 1, arrival_intensities[i]))
    
    while True:
        for task in tasks:
            if random.random() < task.service_intensity:
                if len(task_queue.items) < task_queue.capacity:
                    task_queue.put(task)
                    print(f"Задание {task.task_id} поступило в очередь в {env.now}")
                else:
                    print(f"Превышение очереди. Задание {task.task_id} отклонено в {env.now}")
        yield env.timeout(1)  # Пауза в 1 единицу времени


def cpu(env, cpu_intensities, task_queue):
    cpu_tasks_serviced = 0
    cpu_load = 0
    
    while True:
        if not task_queue.items:
            yield env.timeout(1)  # Pause for 1 unit of time
            cpu_load -= 1
        else:
            current_task = (yield task_queue.get())
            service_time = random.expovariate(cpu_intensities[current_task.task_id - 1])
            yield env.timeout(service_time)
            
            cpu_load += service_time
            cpu_tasks_serviced += 1
            print(f"Task {current_task.task_id} serviced by CPU at {env.now}, service time: {service_time}")


def gpu(env, gpu_intensities, task_queue):
    gpu_tasks_serviced = 0
    gpu_load = 0
    
    while True:
        if not task_queue.items:
            yield env.timeout(1)  # Pause for 1 unit of time
            gpu_load -= 1
        else:
            current_task = (yield task_queue.get())
            service_time = random.expovariate(gpu_intensities[current_task.task_id - 1])
            yield env.timeout(service_time)
            
            gpu_load += service_time
            gpu_tasks_serviced += 1
            print(f"Task {current_task.task_id} serviced by GPU at {env.now}, service time: {service_time}")

def simulate():
    arrival_intensities = [0.6, 0.7, 0.6, 0.3]
    cpu_intensities = [0.7, 0.5, 0.4, 0.3]
    gpu_intensities = [1.4, 1.2, 0.8, 0.4]
    total_time = 1000

    queue_capacity = 30
    env = simpy.Environment()
    task_queue = simpy.Store(env, capacity=queue_capacity)

    env.process(task_generator(env, arrival_intensities, task_queue))
    env.process(cpu(env, cpu_intensities, task_queue))
    env.process(gpu(env, gpu_intensities, task_queue))

    env.run(until=total_time)

results = simulate()
print(results)

Задание 2 поступило в очередь в 0
Task 2 serviced by CPU at 0.8981017151929533, service time: 0.8981017151929533
Задание 1 поступило в очередь в 1
Задание 2 поступило в очередь в 1
Task 1 serviced by GPU at 1.8134842028572822, service time: 0.8134842028572822
Задание 1 поступило в очередь в 2
Задание 2 поступило в очередь в 2
Задание 4 поступило в очередь в 2
Task 2 serviced by GPU at 2.8819519702112952, service time: 1.068467767354013
Задание 2 поступило в очередь в 3
Задание 3 поступило в очередь в 3
Task 1 serviced by GPU at 3.501749519209305, service time: 0.6197975489980101
Task 2 serviced by CPU at 3.6811065412942456, service time: 0.7830048261012924
Задание 2 поступило в очередь в 4
Задание 3 поступило в очередь в 4
Task 4 serviced by GPU at 4.269927979023195, service time: 0.7681784598138902
Задание 1 поступило в очередь в 5
Task 2 serviced by CPU at 5.383724621522202, service time: 1.702618080227956
Task 3 serviced by GPU at 5.613880906895647, service time: 1.3439529278724511


In [51]:
#14cpu 23gpu
import random
import simpy

class Task:
    def __init__(self, task_id, priority, service_intensity):
        self.task_id = task_id
        self.priority = priority
        self.service_intensity = service_intensity


def task_generator(env, arrival_intensities, task_queue):
    tasks = []
    for i in range(len(arrival_intensities)):
        tasks.append(Task(i + 1, i + 1, arrival_intensities[i]))
    
    while True:
        for task in tasks:
            if random.random() < task.service_intensity:
                if len(task_queue.items) < task_queue.capacity:
                    task_queue.put(task)
                    print(f"Task {task.task_id} arrived in the queue at {env.now} with priority {task.priority}")
                else:
                    print(f"Queue overflow. Task {task.task_id} rejected at {env.now}")
        yield env.timeout(1)  # Pause for 1 unit of time


def cpu(env, cpu_intensities, task_queue):
    cpu_tasks_serviced = 0
    cpu_load = 0
    
    while True:
        if not task_queue.items:
            yield env.timeout(1)  # Pause for 1 unit of time
        else:
            highest_priority = min([task.priority for task in task_queue.items])
            # Get all tasks with the highest priority
            highest_priority_tasks = [task for task in task_queue.items if task.priority == highest_priority]

            task = random.choice(highest_priority_tasks)  # Randomly select a task with the highest priority
            service_time = random.expovariate(cpu_intensities[task.task_id - 1])
            yield env.timeout(service_time)
            
            if task in task_queue.items:
                task_queue.items.remove(task)
                cpu_load += service_time
                cpu_tasks_serviced += 1
                print(f"Task {task.task_id} serviced by CPU at {env.now}, service time: {service_time}")

def gpu(env, gpu_intensities, task_queue):
    gpu_tasks_serviced = 0
    gpu_load = 0
    
    while True:
        if not task_queue.items:
            yield env.timeout(1)  # Pause for 1 unit of time
        else:
            highest_priority = min([task.priority for task in task_queue.items])
            # Get all tasks with the highest priority
            highest_priority_tasks = [task for task in task_queue.items if task.priority == highest_priority]

            task = random.choice(highest_priority_tasks)  # Randomly select a task with the highest priority
            service_time = random.expovariate(gpu_intensities[task.task_id - 1])
            yield env.timeout(service_time)
            
            if task in task_queue.items:
                task_queue.items.remove(task)
                gpu_load += service_time
                gpu_tasks_serviced += 1
                print(f"Task {task.task_id} serviced by GPU at {env.now}, service time: {service_time}")
                
def simulate():
    arrival_intensities = [0.6, 0.7, 0.6, 0.3]
    cpu_intensities = [0.7, 0.5, 0.4, 0.3]
    gpu_intensities = [1.4, 1.2, 0.8, 0.4]
    total_time = 1000

    queue_capacity = 30
    env = simpy.Environment()
    task_queue = simpy.Store(env, capacity=queue_capacity)

    env.process(task_generator(env, arrival_intensities, task_queue))
    env.process(cpu(env, cpu_intensities, task_queue))
    env.process(gpu(env, gpu_intensities, task_queue))

    env.run(until=total_time)

results = simulate()
print(results)

Task 3 arrived in the queue at 0 with priority 3
Task 1 arrived in the queue at 1 with priority 1
Task 3 arrived in the queue at 1 with priority 3
Task 3 serviced by GPU at 1.8392241192400745, service time: 1.8392241192400745
Task 1 serviced by GPU at 1.891581456558149, service time: 0.052357337318074536
Task 2 arrived in the queue at 2 with priority 2
Task 3 arrived in the queue at 2 with priority 3
Task 3 serviced by GPU at 2.203322065389615, service time: 0.3117406088314661
Task 2 serviced by GPU at 2.3955289088184304, service time: 0.19220684342881542
Task 3 arrived in the queue at 3 with priority 3
Task 3 serviced by GPU at 3.690649880180975, service time: 1.2951209713625447
Task 2 arrived in the queue at 4 with priority 2
Task 3 serviced by GPU at 4.537599122682444, service time: 0.8469492425014687
Task 1 arrived in the queue at 5 with priority 1
Task 2 arrived in the queue at 5 with priority 2
Task 4 arrived in the queue at 5 with priority 4
Task 2 serviced by GPU at 5.012788147